# Tokyo Rent Prediction - Deep Learning Model Prediction Notebook

This notebook demonstrates how to use trained deep learning models for rent prediction.

## Contents
1. Imports and Setup
2. Load Model
3. Single Prediction Example
4. Batch Prediction Example
5. Model Comparison (Basic vs Attention)
6. Interactive Predictor UI
7. Ward Analysis (Embeddings and Attention Weights)

## 1. Imports and Setup

In [ ]:
# 日本語フォント設定
from rent_utils import setup_japanese_font
setup_japanese_font()

print("✅ 日本語フォント設定完了")

import pandas as pd
import numpy as np
import torch

import seaborn as sns
from sklearn.decomposition import PCA
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Import prediction module
from rent_dl_predict import DeepLearningRentPredictor

import matplotlib.pyplot as plt
# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ All modules imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device available: {'GPU' if torch.cuda.is_available() else 'CPU'}")

## 2. Load Model

Load the trained deep learning model for predictions.

In [ ]:
# Load Attention model (recommended)
predictor_attention = DeepLearningRentPredictor(model_type='attention')

# Load Basic model for comparison
predictor_basic = DeepLearningRentPredictor(model_type='basic')

print("\n" + "="*60)
print("Models loaded successfully!")
print("="*60)

## 3. Single Prediction Example

Predict rent for a single property.

In [ ]:
# Example property
ward = '新宿区'
room_size = 30  # m²
station_distance = 5  # minutes
building_age = 10  # years
structure = 'RC造'
property_type = 'マンション'

# Make prediction
prediction = predictor_attention.predict(
    ward=ward,
    room_size=room_size,
    station_distance=station_distance,
    building_age=building_age,
    structure=structure,
    property_type=property_type
)

# Display result
print("\n" + "="*60)
print("🏠 Single Property Prediction")
print("="*60)
print(f"Ward: {ward}")
print(f"Room Size: {room_size} m²")
print(f"Station Distance: {station_distance} min walk")
print(f"Building Age: {building_age} years")
print(f"Structure: {structure}")
print(f"Property Type: {property_type}")
print("="*60)
print(f"\n💰 Predicted Monthly Rent: ¥{prediction:,.0f}")
print("="*60)

## 4. Batch Prediction Example

Predict rent for multiple properties at once.

In [ ]:
# Define multiple properties
properties = [
    {
        'ward': '港区',
        'room_size': 40,
        'station_distance': 3,
        'building_age': 5,
        'structure': 'SRC造',
        'property_type': 'マンション'
    },
    {
        'ward': '足立区',
        'room_size': 25,
        'station_distance': 10,
        'building_age': 20,
        'structure': '木造',
        'property_type': 'アパート'
    },
    {
        'ward': '渋谷区',
        'room_size': 35,
        'station_distance': 7,
        'building_age': 12,
        'structure': 'RC造',
        'property_type': 'マンション'
    },
    {
        'ward': '中野区',
        'room_size': 28,
        'station_distance': 8,
        'building_age': 15,
        'structure': '鉄骨造',
        'property_type': 'ハイツ'
    }
]

# Batch prediction
predictions = predictor_attention.batch_predict(properties)

# Display results
print("\n" + "="*60)
print("📊 Batch Prediction Results")
print("="*60)

for i, (prop, pred) in enumerate(zip(properties, predictions), 1):
    print(f"\nProperty {i}:")
    print(f"  Ward: {prop['ward']}")
    print(f"  Size: {prop['room_size']}m², Station: {prop['station_distance']}min, Age: {prop['building_age']}yr")
    print(f"  Structure: {prop['structure']}, Type: {prop['property_type']}")
    print(f"  → Predicted Rent: ¥{pred:,.0f}")

print("\n" + "="*60)
print(f"Average Predicted Rent: ¥{np.mean(predictions):,.0f}")
print(f"Highest: ¥{max(predictions):,.0f}")
print(f"Lowest: ¥{min(predictions):,.0f}")
print("="*60)

## 5. Model Comparison (Basic vs Attention)

Compare predictions between Basic and Attention models.

In [ ]:
# 日本語フォント再設定
from rent_utils import setup_japanese_font
setup_japanese_font()

# Test wards
test_wards = ['港区', '千代田区', '新宿区', '中野区', '練馬区', '足立区']

# Fixed conditions
test_conditions = {
    'room_size': 30,
    'station_distance': 5,
    'building_age': 10,
    'structure': 'RC造',
    'property_type': 'マンション'
}

# Compare predictions
results = []
for ward in test_wards:
    pred_basic = predictor_basic.predict(ward=ward, **test_conditions)
    pred_attention = predictor_attention.predict(ward=ward, **test_conditions)
    
    results.append({
        'Ward': ward,
        'Basic Model': pred_basic,
        'Attention Model': pred_attention,
        'Difference': pred_attention - pred_basic
    })

# Create DataFrame
df_comparison = pd.DataFrame(results)

# Display table
print("\n" + "="*80)
print("🔄 Model Comparison: Basic vs Attention")
print("="*80)
print(f"Test Conditions: {test_conditions['room_size']}m², {test_conditions['station_distance']}min walk, ")
print(f"                 {test_conditions['building_age']}yr old, {test_conditions['structure']}, {test_conditions['property_type']}")
print("="*80)
print(df_comparison.to_string(index=False))
print("="*80)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart comparison
x = np.arange(len(test_wards))
width = 0.35

axes[0].bar(x - width/2, df_comparison['Basic Model'], width, label='Basic Model', alpha=0.8)
axes[0].bar(x + width/2, df_comparison['Attention Model'], width, label='Attention Model', alpha=0.8)
axes[0].set_xlabel('Ward')
axes[0].set_ylabel('Predicted Rent (¥)')
axes[0].set_title('Model Comparison by Ward')
axes[0].set_xticks(x)
axes[0].set_xticklabels(test_wards, rotation=45, ha='right')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Scatter plot
axes[1].scatter(df_comparison['Basic Model'], df_comparison['Attention Model'], 
                s=100, alpha=0.6, edgecolors='black', linewidth=1.5)
axes[1].plot([df_comparison['Basic Model'].min(), df_comparison['Basic Model'].max()],
             [df_comparison['Basic Model'].min(), df_comparison['Basic Model'].max()],
             'r--', alpha=0.5, label='Perfect Agreement')

for i, ward in enumerate(test_wards):
    axes[1].annotate(ward, 
                    (df_comparison['Basic Model'].iloc[i], df_comparison['Attention Model'].iloc[i]),
                    fontsize=9, ha='left', va='bottom')

axes[1].set_xlabel('Basic Model Prediction (¥)')
axes[1].set_ylabel('Attention Model Prediction (¥)')
axes[1].set_title('Basic vs Attention Model Predictions')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# Statistics
print("\n📈 Comparison Statistics:")
print(f"Mean Absolute Difference: ¥{abs(df_comparison['Difference']).mean():,.0f}")
print(f"Correlation: {df_comparison['Basic Model'].corr(df_comparison['Attention Model']):.4f}")

## 6. Interactive Predictor UI

Interactive widget-based interface for rent prediction.

In [ ]:
class InteractiveRentPredictorDL:
    """Deep Learning-based Interactive Rent Prediction UI"""
    
    def __init__(self):
        """Initialize UI and settings"""
        self.predictor_basic = None
        self.predictor_attention = None
        self.current_predictor = None
        self.setup_ward_info()
        self.setup_ui_widgets()
        self.setup_event_handlers()
        # Load Attention model by default
        self.load_predictors('attention')
    
    def setup_ward_info(self):
        """Setup ward information"""
        self.ward_list = [
            ('港区', '超高級', '#FF1744', 200000),
            ('千代田区', '超高級', '#FF1744', 180000),
            ('中央区', '超高級', '#FF1744', 160000),
            ('渋谷区', '超高級', '#FF1744', 150000),
            ('目黒区', '高級', '#FF6F00', 130000),
            ('文京区', '高級', '#FF6F00', 125000),
            ('新宿区', '高級', '#FF6F00', 120000),
            ('品川区', '高級', '#FF6F00', 115000),
            ('世田谷区', '高級', '#FF6F00', 110000),
            ('豊島区', '中価格', '#2196F3', 90000),
            ('台東区', '中価格', '#2196F3', 85000),
            ('中野区', '中価格', '#2196F3', 85000),
            ('杉並区', '中価格', '#2196F3', 85000),
            ('江東区', '中価格', '#2196F3', 80000),
            ('大田区', '中価格', '#2196F3', 80000),
            ('墨田区', '中価格', '#2196F3', 75000),
            ('練馬区', '中価格', '#2196F3', 75000),
            ('北区', '低価格', '#4CAF50', 60000),
            ('板橋区', '低価格', '#4CAF50', 55000),
            ('荒川区', '低価格', '#4CAF50', 50000),
            ('江戸川区', '低価格', '#4CAF50', 50000),
            ('葛飾区', '低価格', '#4CAF50', 45000),
            ('足立区', '低価格', '#4CAF50', 40000)
        ]
    
    def setup_ui_widgets(self):
        """Create UI widgets"""
        style = {'description_width': '140px'}
        layout_long = widgets.Layout(width='550px')
        layout_short = widgets.Layout(width='350px')

        # Model selector
        self.model_selector = widgets.RadioButtons(
            options=[('Basic Model', 'basic'), ('Attention Model (Recommended)', 'attention')],
            value='attention',
            description='Model:',
            style=style,
            layout=layout_long
        )

        # Input widgets
        self.ward_dropdown = widgets.Dropdown(
            options=[f"{ward} ({level})" for ward, level, _, _ in self.ward_list],
            value='新宿区 (高級)',
            description='Ward:',
            style=style,
            layout=layout_long
        )
        
        self.room_size_slider = widgets.IntSlider(
            value=30, min=15, max=100, step=5,
            description='Room Size:',
            style=style,
            layout=layout_long
        )
        self.room_size_label = widgets.Label(value='30 m²')
        
        self.station_distance_slider = widgets.IntSlider(
            value=5, min=1, max=20, step=1,
            description='Station Distance:',
            style=style,
            layout=layout_long
        )
        self.station_label = widgets.Label(value='5 min')
        
        self.building_age_slider = widgets.IntSlider(
            value=10, min=0, max=50, step=1,
            description='Building Age:',
            style=style,
            layout=layout_long
        )
        self.age_label = widgets.Label(value='10 years')
        
        self.structure_dropdown = widgets.Dropdown(
            options=['木造', 'RC造', '鉄骨造', 'SRC造'],
            value='RC造',
            description='Structure:',
            style=style,
            layout=layout_short
        )
        
        self.property_type_dropdown = widgets.Dropdown(
            options=['マンション', 'アパート', 'ハイツ', 'コーポ'],
            value='マンション',
            description='Property Type:',
            style=style,
            layout=layout_short
        )
        
        # Mode tabs
        self.mode_tabs = widgets.Tab()
        self.single_mode = widgets.VBox([widgets.HTML("<p>Predict rent for a single property</p>")])
        self.compare_mode = widgets.VBox([widgets.HTML("<p>Compare rent across multiple wards</p>")])
        self.analysis_mode = widgets.VBox([widgets.HTML("<p>Analyze AI model internals</p>")])
        
        self.mode_tabs.children = [self.single_mode, self.compare_mode, self.analysis_mode]
        self.mode_tabs.set_title(0, 'Single Prediction')
        self.mode_tabs.set_title(1, 'Ward Comparison')
        self.mode_tabs.set_title(2, 'AI Analysis')
        
        # Predict button
        self.predict_button = widgets.Button(
            description='Run AI Prediction',
            button_style='primary',
            tooltip='Predict with deep learning',
            layout=widgets.Layout(width='200px', height='45px')
        )
        
        # Output area
        self.output = widgets.Output()
    
    def setup_event_handlers(self):
        """Setup event handlers"""
        # Slider label updates
        self.room_size_slider.observe(self._update_room_label, names='value')
        self.station_distance_slider.observe(self._update_station_label, names='value')
        self.building_age_slider.observe(self._update_age_label, names='value')

        # Model selector change
        self.model_selector.observe(self._on_model_change, names='value')

        # Predict button
        self.predict_button.on_click(self.on_predict_click)

    def load_predictors(self, model_type):
        """Load predictor"""
        try:
            if model_type == 'basic':
                if self.predictor_basic is None:
                    self.predictor_basic = DeepLearningRentPredictor(model_type='basic')
                self.current_predictor = self.predictor_basic
            else:  # 'attention'
                if self.predictor_attention is None:
                    self.predictor_attention = DeepLearningRentPredictor(model_type='attention')
                self.current_predictor = self.predictor_attention
        except Exception as e:
            print(f"Model loading error: {e}")

    def _on_model_change(self, change):
        """Handle model selector change"""
        self.load_predictors(change['new'])
    
    def _update_room_label(self, change):
        self.room_size_label.value = f'{change["new"]} m²'
    
    def _update_station_label(self, change):
        self.station_label.value = f'{change["new"]} min'
    
    def _update_age_label(self, change):
        self.age_label.value = f'{change["new"]} years'
    
    def on_predict_click(self, b):
        """Predict button click handler"""
        with self.output:
            clear_output()
            
            current_tab = self.mode_tabs.selected_index
            
            if current_tab == 0:  # Single prediction
                self.single_prediction()
            elif current_tab == 1:  # Ward comparison
                self.comparison_prediction()
            elif current_tab == 2:  # AI analysis
                self.ai_analysis()
    
    def single_prediction(self):
        """Single property prediction"""
        ward = self.ward_dropdown.value.split(' (')[0]

        if self.current_predictor is None:
            print("Model not loaded")
            return

        prediction = self.current_predictor.predict(
            ward,
            self.room_size_slider.value,
            self.station_distance_slider.value,
            self.building_age_slider.value,
            self.structure_dropdown.value,
            self.property_type_dropdown.value
        )
        
        # Ward info
        ward_info = next((info for info in self.ward_list if info[0] == ward), None)
        if not ward_info:
            return
        
        ward_level = ward_info[1]
        bg_color = ward_info[2]
        base_price = ward_info[3]
        
        # HTML output
        html = f"""
        <div style="font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;">
            <div style="background: linear-gradient(135deg, {bg_color}15, white); 
                        border: 3px solid {bg_color}; border-radius: 20px; 
                        padding: 30px; box-shadow: 0 10px 30px rgba(0,0,0,0.1);">
                
                <h2 style="color: {bg_color}; margin: 0 0 25px 0; font-size: 28px;">
                    Deep Learning Prediction Result
                </h2>
                
                <div style="background: white; padding: 20px; border-radius: 12px; 
                            margin-bottom: 20px; box-shadow: 0 2px 10px rgba(0,0,0,0.05);">
                    <h3 style="color: #555; margin: 0 0 15px 0; font-size: 16px;">Input Conditions</h3>
                    <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 12px; font-size: 14px;">
                        <div><strong>Ward:</strong> {ward} 
                            <span style="background: {bg_color}; color: white; padding: 3px 10px; 
                                       border-radius: 15px; font-size: 11px;">{ward_level}</span>
                        </div>
                        <div><strong>Size:</strong> {self.room_size_slider.value} m²</div>
                        <div><strong>Station:</strong> {self.station_distance_slider.value} min walk</div>
                        <div><strong>Age:</strong> {self.building_age_slider.value} years</div>
                        <div><strong>Structure:</strong> {self.structure_dropdown.value}</div>
                        <div><strong>Type:</strong> {self.property_type_dropdown.value}</div>
                    </div>
                </div>
                
                <div style="background: linear-gradient(135deg, {bg_color}, {bg_color}dd); 
                            color: white; padding: 30px; border-radius: 15px; 
                            text-align: center;">
                    <div style="font-size: 14px; opacity: 0.9; margin-bottom: 10px;">
                        AI PREDICTED MONTHLY RENT
                    </div>
                    <div style="font-size: 56px; font-weight: 300; margin: 15px 0;">
                        ¥{prediction:,.0f}
                    </div>
                    <div style="font-size: 12px; opacity: 0.7;">
                        Ward Base Price: ¥{base_price:,} | 
                        Difference: ¥{prediction - base_price:+,.0f}
                    </div>
                </div>
                
                <div style="margin-top: 20px; padding: 15px; background: #f8f9fa;
                            border-radius: 8px; font-size: 12px; color: #666;">
                    <strong>Model Info:</strong>
                    {self.current_predictor.model_type.upper()} Model |
                    PyTorch Neural Network |
                    {'Attention Mechanism | Ward Embedding (32d) | 3 Layers (512→256→128)' if self.current_predictor.model_type == 'attention' else 'Ward Embedding (16d) | 3 Layers (256→128→64)'} |
                    Batch Normalization | Dropout
                </div>
            </div>
        </div>
        """
        
        display(HTML(html))
    
    def comparison_prediction(self):
        """Multi-ward comparison prediction"""
        if self.current_predictor is None:
            print("Model not loaded")
            return

        comparison_wards = [
            '港区', '千代田区', '中央区', '渋谷区',
            '新宿区', '世田谷区', '中野区', '練馬区',
            '板橋区', '足立区'
        ]

        predictions = []
        for ward in comparison_wards:
            pred = self.current_predictor.predict(
                ward,
                self.room_size_slider.value,
                self.station_distance_slider.value,
                self.building_age_slider.value,
                self.structure_dropdown.value,
                self.property_type_dropdown.value
            )
            
            ward_info = next((info for info in self.ward_list if info[0] == ward), None)
            if ward_info:
                predictions.append({
                    'ward': ward,
                    'prediction': pred,
                    'level': ward_info[1],
                    'color': ward_info[2],
                    'base': ward_info[3]
                })
        
        predictions.sort(key=lambda x: x['prediction'], reverse=True)
        max_pred = predictions[0]['prediction'] if predictions else 1
        
        # HTML output
        html = f"""
        <div style="font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;">
            <div style="background: white; border: 2px solid #5e72e4; border-radius: 20px; 
                        padding: 30px; box-shadow: 0 10px 30px rgba(0,0,0,0.1);">
                
                <h2 style="color: #5e72e4; margin: 0 0 25px 0;">Ward Comparison - Deep Learning</h2>
                
                <div style="background: #f8f9fa; padding: 15px; border-radius: 10px; 
                            margin-bottom: 25px; font-size: 14px;">
                    <strong>Conditions:</strong>
                    {self.room_size_slider.value}m² | Station {self.station_distance_slider.value}min | 
                    Age {self.building_age_slider.value}yr | {self.structure_dropdown.value} | 
                    {self.property_type_dropdown.value}
                </div>
                
                <div style="margin-bottom: 25px;">
        """
        
        for i, pred_info in enumerate(predictions):
            bar_width = (pred_info['prediction'] / max_pred) * 100 if max_pred > 0 else 0
            diff_from_base = pred_info['prediction'] - pred_info['base']
            
            html += f"""
                <div style="display: flex; align-items: center; margin: 15px 0;">
                    <div style="width: 30px; text-align: center; font-weight: bold; color: #999;">
                        {i+1}
                    </div>
                    <div style="width: 100px; font-weight: 600; margin-left: 10px;">
                        {pred_info['ward']}
                    </div>
                    <div style="flex: 1; margin: 0 20px;">
                        <div style="background: #e9ecef; border-radius: 25px; height: 32px; overflow: hidden;">
                            <div style="width: {bar_width}%; 
                                        background: {pred_info['color']}; 
                                        height: 100%; display: flex; align-items: center; 
                                        justify-content: flex-end; padding-right: 15px;">
                                <span style="color: white; font-weight: 600;">
                                    ¥{pred_info['prediction']:,.0f}
                                </span>
                            </div>
                        </div>
                    </div>
                    <div style="text-align: right; min-width: 80px; font-size: 11px; color: #666;">
                        {diff_from_base:+,.0f}
                    </div>
                    <span style="background: {pred_info['color']}; color: white; 
                                padding: 5px 12px; border-radius: 20px; font-size: 11px; 
                                min-width: 60px; text-align: center; margin-left: 10px;">
                        {pred_info['level']}
                    </span>
                </div>
            """
        
        # Statistics
        if len(predictions) > 1:
            diff = predictions[0]['prediction'] - predictions[-1]['prediction']
            avg_pred = sum(p['prediction'] for p in predictions) / len(predictions)
            
            html += f"""
                </div>
                
                <div style="margin-top: 30px; padding: 20px; 
                            background: linear-gradient(135deg, #667eea, #764ba2); 
                            border-radius: 12px; color: white;">
                    <h4 style="margin: 0 0 15px 0;">AI Analysis Results</h4>
                    <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 20px;">
                        <div style="text-align: center;">
                            <div style="font-size: 12px; opacity: 0.8;">Highest</div>
                            <div style="font-size: 20px; font-weight: bold;">¥{predictions[0]['prediction']:,.0f}</div>
                            <div style="font-size: 10px; opacity: 0.7;">{predictions[0]['ward']}</div>
                        </div>
                        <div style="text-align: center;">
                            <div style="font-size: 12px; opacity: 0.8;">Average</div>
                            <div style="font-size: 20px; font-weight: bold;">¥{avg_pred:,.0f}</div>
                            <div style="font-size: 10px; opacity: 0.7;">All {len(predictions)} wards</div>
                        </div>
                        <div style="text-align: center;">
                            <div style="font-size: 12px; opacity: 0.8;">Lowest</div>
                            <div style="font-size: 20px; font-weight: bold;">¥{predictions[-1]['prediction']:,.0f}</div>
                            <div style="font-size: 10px; opacity: 0.7;">{predictions[-1]['ward']}</div>
                        </div>
                    </div>
                </div>
            </div>
        </div>
        """
        
        display(HTML(html))
    
    def ai_analysis(self):
        """AI model internal analysis"""
        if self.current_predictor is None:
            print("Model not loaded")
            return
            
        embeddings, ward_names = self.current_predictor.get_ward_embeddings()
        
        if embeddings is None:
            display(HTML("<p>Model not loaded</p>"))
            return
        
        # Ward embedding importance
        importance_scores = np.linalg.norm(embeddings, axis=1)
        ward_importance = list(zip(ward_names, importance_scores))
        ward_importance.sort(key=lambda x: x[1], reverse=True)
        
        html = """
        <div style="font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;">
            <div style="background: white; border: 2px solid #6c5ce7; border-radius: 20px; 
                        padding: 30px; box-shadow: 0 10px 30px rgba(0,0,0,0.1);">
                
                <h2 style="color: #6c5ce7; margin: 0 0 25px 0;">AI Model Internal Analysis</h2>
                
                <div style="background: #f8f9fa; padding: 20px; border-radius: 12px; margin-bottom: 25px;">
                    <h3 style="color: #495057; margin: 0 0 15px 0; font-size: 18px;">
                        Ward Embedding Importance (Top 10)
                    </h3>
                    <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 15px;">
        """
        
        max_importance = ward_importance[0][1] if ward_importance else 1
        
        for i, (ward, score) in enumerate(ward_importance[:10]):
            ward_info = next((info for info in self.ward_list if info[0] == ward), None)
            color = ward_info[2] if ward_info else '#666'
            bar_width = (score / max_importance) * 100
            
            html += f"""
                        <div style="display: flex; align-items: center;">
                            <div style="width: 80px; font-weight: 600;">{i+1}. {ward}</div>
                            <div style="flex: 1; margin: 0 10px;">
                                <div style="background: #e9ecef; border-radius: 10px; height: 20px; overflow: hidden;">
                                    <div style="width: {bar_width}%; background: {color}; height: 100%; 
                                                display: flex; align-items: center; justify-content: flex-end; padding-right: 8px;">
                                        <span style="color: white; font-size: 11px; font-weight: bold;">{score:.2f}</span>
                                    </div>
                                </div>
                            </div>
                        </div>
            """
        
        html += """
                    </div>
                </div>
        """
        
        # Attention weights (if attention model)
        if self.current_predictor.model_type == 'attention':
            html += """
                <div style="background: #f8f9fa; padding: 20px; border-radius: 12px;">
                    <h3 style="color: #495057; margin: 0 0 15px 0; font-size: 18px;">
                        Attention Mechanism Analysis
                    </h3>
            """
            
            # Get attention weights for top wards
            attention_data = []
            for ward, _ in ward_importance[:5]:
                attn_weights = self.current_predictor.get_attention_weights(ward)
                if attn_weights is not None:
                    attention_data.append((ward, float(attn_weights[0])))
            
            if attention_data:
                max_attn = max(a[1] for a in attention_data)
                
                for ward, attn in attention_data:
                    ward_info = next((info for info in self.ward_list if info[0] == ward), None)
                    color = ward_info[2] if ward_info else '#666'
                    bar_width = (attn / max_attn) * 100 if max_attn > 0 else 0
                    
                    html += f"""
                    <div style="display: flex; align-items: center; margin: 10px 0;">
                        <div style="width: 100px; font-weight: 600;">{ward}</div>
                        <div style="flex: 1; margin: 0 10px;">
                            <div style="background: #e9ecef; border-radius: 10px; height: 24px; overflow: hidden;">
                                <div style="width: {bar_width}%; background: {color}; height: 100%; 
                                            display: flex; align-items: center; justify-content: flex-end; padding-right: 10px;">
                                    <span style="color: white; font-size: 12px; font-weight: bold;">{attn:.4f}</span>
                                </div>
                            </div>
                        </div>
                    </div>
                    """
            
            html += "</div>"
        
        # Model info
        html += f"""
                <div style="margin-top: 25px; padding: 15px; background: #6c5ce7; color: white; border-radius: 8px;">
                    <strong>Model Architecture:</strong><br>
                    Type: {self.current_predictor.model_type.upper()}<br>
                    Embedding Dimension: {'32' if self.current_predictor.model_type == 'attention' else '16'}<br>
                    Hidden Layers: {'512→256→128' if self.current_predictor.model_type == 'attention' else '256→128→64'}<br>
                    Features: Ward Embeddings, Structure Embeddings, Type Embeddings, Numeric Features
                </div>
            </div>
        </div>
        """
        
        display(HTML(html))
    
    def show(self):
        """Display the UI"""
        # Input section
        input_section = widgets.VBox([
            widgets.HTML("<h3 style='color: #5e72e4;'>Property Information Input</h3>"),
            self.model_selector,
            widgets.HTML("<div style='margin: 10px 0; border-top: 1px solid #dee2e6;'></div>"),
            self.ward_dropdown,
            widgets.HBox([self.room_size_slider, self.room_size_label]),
            widgets.HBox([self.station_distance_slider, self.station_label]),
            widgets.HBox([self.building_age_slider, self.age_label]),
            widgets.HBox([self.structure_dropdown, self.property_type_dropdown]),
            widgets.HTML("<div style='margin: 20px 0; border-top: 1px solid #dee2e6;'></div>"),
            self.mode_tabs,
        ])
        
        # Main layout
        main_layout = widgets.VBox([
            widgets.HTML("<h2 style='color: #5e72e4;'>Deep Learning Rent Prediction System</h2>"),
            input_section,
            widgets.HTML("<div style='margin: 20px 0;'></div>"),
            widgets.HBox([self.predict_button]),
            widgets.HTML("<div style='margin: 10px 0;'></div>"),
            self.output
        ])
        
        display(main_layout)

# Create and display the interactive predictor
print("Initializing Interactive Predictor...")
predictor_ui = InteractiveRentPredictorDL()
predictor_ui.show()

## 7. Ward Analysis (Embeddings and Attention Weights)

Analyze ward embeddings and attention mechanism.

### 7.1 Ward Embeddings Visualization

In [ ]:
# 日本語フォント再設定
from rent_utils import setup_japanese_font
setup_japanese_font()

# Get ward embeddings
embeddings, ward_names = predictor_attention.get_ward_embeddings()

print(f"Embedding shape: {embeddings.shape}")
print(f"Number of wards: {len(ward_names)}")
print(f"Embedding dimension: {embeddings.shape[1]}")

# PCA projection to 2D
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings)

# Load data for ward prices
df = pd.read_csv('tokyo_rent_data_v2.csv')
ward_prices = [df[df['区'] == ward]['家賃_円'].mean() for ward in ward_names]

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 2D embedding scatter plot
scatter = axes[0].scatter(embeddings_2d[:, 0], embeddings_2d[:, 1],
                          c=ward_prices, cmap='RdYlBu_r',
                          s=200, alpha=0.7, edgecolors='black', linewidth=1.5)

# Add ward labels
for i, ward in enumerate(ward_names):
    axes[0].annotate(ward, (embeddings_2d[i, 0], embeddings_2d[i, 1]),
                     fontsize=9, ha='center', va='bottom',
                     bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))

axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontsize=12)
axes[0].set_title('Ward Embeddings (PCA 2D Projection)', fontsize=14, fontweight='bold')
plt.colorbar(scatter, ax=axes[0], label='Average Rent (¥)')
axes[0].grid(True, alpha=0.3)

# Embedding importance
importance = np.linalg.norm(embeddings, axis=1)
sorted_indices = np.argsort(importance)[::-1]

top_n = 15
top_wards = [ward_names[i] for i in sorted_indices[:top_n]]
top_importance = [importance[i] for i in sorted_indices[:top_n]]

bars = axes[1].barh(range(top_n), top_importance, color='skyblue', edgecolor='black', linewidth=1.5)
axes[1].set_yticks(range(top_n))
axes[1].set_yticklabels(top_wards)
axes[1].set_xlabel('Embedding Norm (Importance)', fontsize=12)
axes[1].set_title('Ward Embedding Importance (Top 15)', fontsize=14, fontweight='bold')
axes[1].invert_yaxis()
axes[1].grid(True, alpha=0.3, axis='x')

# Add value labels
for i, (bar, val) in enumerate(zip(bars, top_importance)):
    axes[1].text(val, i, f' {val:.2f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('ward_embeddings_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nTop 5 Most Important Wards (by embedding norm):")
for i in range(5):
    print(f"{i+1}. {ward_names[sorted_indices[i]]}: {importance[sorted_indices[i]]:.3f}")

### 7.2 Attention Weights Analysis (Attention Model Only)

In [ ]:
# 日本語フォント再設定
from rent_utils import setup_japanese_font
setup_japanese_font()

if predictor_attention.model_type == 'attention':
    # Collect attention weights for all wards
    attention_weights_all = []
    
    for ward in ward_names:
        attn_weights = predictor_attention.get_attention_weights(ward)
        if attn_weights is not None:
            attention_weights_all.append(float(attn_weights[0]))
        else:
            attention_weights_all.append(0)
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Attention weights by ward
    sorted_indices = np.argsort(attention_weights_all)[::-1]
    sorted_wards = [ward_names[i] for i in sorted_indices]
    sorted_weights = [attention_weights_all[i] for i in sorted_indices]
    
    colors = plt.cm.RdYlBu_r(np.linspace(0.2, 0.8, len(sorted_weights)))
    bars = axes[0].barh(range(len(sorted_weights)), sorted_weights, color=colors, edgecolor='black', linewidth=1)
    axes[0].set_yticks(range(len(sorted_weights)))
    axes[0].set_yticklabels(sorted_wards, fontsize=9)
    axes[0].set_xlabel('Attention Weight', fontsize=12)
    axes[0].set_title('Attention Weights by Ward', fontsize=14, fontweight='bold')
    axes[0].invert_yaxis()
    axes[0].grid(True, alpha=0.3, axis='x')
    
    # Add value labels
    for i, (bar, val) in enumerate(zip(bars, sorted_weights)):
        axes[0].text(val, i, f' {val:.4f}', va='center', fontsize=8)
    
    # Attention vs Average Price
    axes[1].scatter(ward_prices, attention_weights_all, s=150, alpha=0.7, 
                    edgecolors='black', linewidth=1.5, c=ward_prices, cmap='RdYlBu_r')
    axes[1].set_xlabel('Average Rent (¥)', fontsize=12)
    axes[1].set_ylabel('Attention Weight', fontsize=12)
    axes[1].set_title('Attention Weight vs Average Rent', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    
    # Label key wards
    key_wards = ['港区', '千代田区', '足立区', '葛飾区']
    for i, ward in enumerate(ward_names):
        if ward in key_wards:
            axes[1].annotate(ward, (ward_prices[i], attention_weights_all[i]),
                           fontsize=10, ha='left', va='bottom',
                           bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.6))
    
    # Correlation
    correlation = np.corrcoef(ward_prices, attention_weights_all)[0, 1]
    axes[1].text(0.05, 0.95, f'Correlation: {correlation:.3f}',
                transform=axes[1].transAxes, fontsize=12,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.savefig('attention_weights_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n" + "="*60)
    print("Attention Weights Analysis")
    print("="*60)
    print(f"Correlation with average rent: {correlation:.4f}")
    print(f"\nTop 5 Highest Attention Wards:")
    for i in range(5):
        idx = sorted_indices[i]
        print(f"{i+1}. {ward_names[idx]}: {attention_weights_all[idx]:.4f}")
    print("="*60)
else:
    print("Basic model does not have attention mechanism.")

### 7.3 Embedding Similarity Matrix

In [ ]:
# 日本語フォント再設定
from rent_utils import setup_japanese_font
setup_japanese_font()

# Calculate cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(embeddings)

# Visualization
plt.figure(figsize=(14, 12))
im = plt.imshow(similarity_matrix, cmap='RdBu', vmin=-1, vmax=1)
plt.colorbar(im, label='Cosine Similarity')

plt.xticks(range(len(ward_names)), ward_names, rotation=90, fontsize=9)
plt.yticks(range(len(ward_names)), ward_names, fontsize=9)
plt.title('Ward Embedding Cosine Similarity Matrix', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('ward_similarity_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Find most similar ward pairs
print("\n" + "="*60)
print("Most Similar Ward Pairs (excluding self-similarity)")
print("="*60)

# Get upper triangle indices (excluding diagonal)
triu_indices = np.triu_indices_from(similarity_matrix, k=1)
similarities = similarity_matrix[triu_indices]
sorted_sim_indices = np.argsort(similarities)[::-1]

for i in range(10):
    idx = sorted_sim_indices[i]
    row = triu_indices[0][idx]
    col = triu_indices[1][idx]
    sim = similarities[idx]
    print(f"{i+1}. {ward_names[row]} - {ward_names[col]}: {sim:.4f}")

print("="*60)

## Summary

This notebook demonstrated:

1. **Loading Models**: How to load and use both Basic and Attention deep learning models
2. **Single Predictions**: Making predictions for individual properties
3. **Batch Predictions**: Efficient prediction for multiple properties
4. **Model Comparison**: Comparing predictions between Basic and Attention models
5. **Interactive UI**: Widget-based interface for easy prediction
6. **Ward Analysis**: 
   - Ward embedding visualization and importance
   - Attention weights analysis
   - Ward similarity analysis

The Attention model generally provides more nuanced predictions by learning complex relationships between wards and other features through its attention mechanism.